In [1]:
# Import packages

import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_notebook, output_file
from bokeh.models import ColumnDataSource, LinearColorMapper, CategoricalColorMapper, LogColorMapper, ColorBar, Label
from bokeh.models.tools import HoverTool, ZoomInTool, ZoomOutTool
from bokeh.palettes import Spectral, Viridis256, Cividis256, Magma256


In [2]:
# Get data to plot

RD = pd.read_csv("Rain_Data_with_sewer_characteristics.csv")
OD = pd.read_csv("Other_Data_with_sewer_characteristics.csv")



In [3]:
unique_OD = OD['MATERIAL_SEWER_PIPE'].unique()
print(unique_OD)
print(len(unique_OD))
      
unique_RD = RD.MATERIAL_SEWER_PIPE.unique()
print(unique_RD)
print(len(unique_RD))


['VCP CIP' ' ' 'VC' 'AC' 'VCP' 'clay' 'PVC' 'ACP' 'vcp' 'Cast Iron' 'sdr'
 'CLAY' 'steel' 'Plastic' 'iron' 'C900' 'Concrete' 'Steel' 'pvc' 'c/i'
 'PEP' 'ABS' 'RCCP' 'Clay' 'SDR-35' 'CP' 'VCP/ABS' 'Iron'
 'Brick and cast iron' 'AC Pipe' 'Cast iron' 'SDR' 'PLA' 'CIP' 'RCP' 'abs'
 'ac' 'Ductile Iron' 'cast iron' 'PVC LIner' 'DIP' 'HDPE' 'Debris'
 'V.C.P.' 'PCV' 'Clay pipe' 'PVC Liner' 'CI' 'Vitrified Clay' 'UNK'
 'transite' 'Pl' 'Plastic & Clay' 'plastic' 'SDR 35 PVC'
 'VCP (CIPP Lined)' 'Fiber Conduit' 'Transite' 'Corrugated Steel' '6'
 'SDR 35' 'ORB' 'Asbestos Concrete' 'Liner' 'di' 'ccip' 'Galvanized'
 'poly lined ductile iron' 'conc' 'PVC-3' 'VCP and Steel' 'CON' 'vc'
 'PVC/Cast Iron' 'lined' 'na' 'acp' 'ABCF' 'Plastic liner' 'UKN' 'cly'
 'unk' 'DIP/PVC' 'rcp' 'Clay/PVC' 'Vitrified Clay Pipe'
 'vitrified clay pipe' 'ductile iron pipe' 'Duc Iron' nan 'concrete'
 'Asbestos Cement' 'ductile iron' 'P.E.' 'metal' 'SDR35' 'Lined VCP'
 'hdpe' 'Aluminum' 'RGRCP-PTL' 'RCP-CTL' 'RCP-CTL, FR Pol

In [4]:
# Input Measurement to Plot: SMM

print("(A) Soil Moisture - Surface (0-5cm)")
print("(B) Soil Moisture - Root Zone (0-100cm)")
print("(C) Soil Moisture - Profile (0cm-bedrock)")
print("(D) Soil Water Infiltration Flux")
print("(E) Vegetation Greenness Fraction")
print("(F) Surface Temperature")
print("(G) Surface Pressure")

print("(H) None of the above -- Choose a spill-cause measurement")

good_value = False
value = ""
SMAP = True

while good_value == False :
    choice = input("Choose an SMAP measurement: ")

    try:
        if choice == 'A' or choice == 'a' : SMM = 'sm_surface' # & SMM_units = '(m\u00b3/m\u00b3)'
        if choice == 'B' or choice == 'b' : SMM = 'sm_rootzone'
        if choice == 'C' or choice == 'c' : SMM = 'sm_profile'
        if choice == 'D' or choice == 'd' : SMM = 'soil_water_infiltration_flux'
        if choice == 'E' or choice == 'e' : SMM = 'vegetation_greenness_fraction'
        if choice == 'F' or choice == 'f' : SMM = 'surface_temp'
        if choice == 'G' or choice == 'g' : SMM = 'surface_pressure'
        if choice == 'H' or choice == 'h' : 
            
            SMAP = False
            
            print("\n") 
            print("(1) Spill Category (1, 2 or 3)")
            print("(2) Spill Volume")
            print("(3) Sewer Pipe Diameter")
            print("(4) Sewer Pipe Age")
            
            while good_value == False :
                choice = input("Choose a spill-cause measurement: ")
                try:
                    if choice == '1' : SMM = 'SPILL_TYPE'
                    if choice == '2' : SMM = 'SPILL_VOL'
                    if choice == '3' : SMM = 'DIAMETER_OF_SEWER'    
                    if choice == '4' : SMM = 'AGE_SEWER_PIPE'
                    good_value = True

                except ValueError:
                    print("Error: Invalid choice.")
                    
        good_value = True
        
    except ValueError:
        print("Error: Invalid choice.")

print("\n")     
print("Measurement to plot = " + SMM)


(A) Soil Moisture - Surface (0-5cm)
(B) Soil Moisture - Root Zone (0-100cm)
(C) Soil Moisture - Profile (0cm-bedrock)
(D) Soil Water Infiltration Flux
(E) Vegetation Greenness Fraction
(F) Surface Temperature
(G) Surface Pressure
(H) None of the above -- Choose a spill-cause measurement
Choose an SMAP measurement: A


Measurement to plot = sm_surface


In [5]:
# Create Bokeh sources

rain_data = dict(x=RD['x'].values, 
                 y=RD['y'].values, 
                 SID = RD['SPILL ID'].values, 
                 DATE_TIME = RD['start_date'].values, 
                 CAUSE = RD['SPILL CAUSE'].values,
                 CATEGORY = RD['SPILL_TYPE'].values,
                 VOLUME = RD['SPILL_VOL'].values,
                 MATERIAL = RD['MATERIAL_SEWER_PIPE'].values,
                 AGE = RD['AGE_SEWER_PIPE'].values,
                 DIAMETER = RD['DIAMETER_OF_SEWER'].values,
                 PLOT = RD[SMM].values)

rain_source = ColumnDataSource(rain_data)
    
other_data = dict(x=OD['x'].values, 
                 y=OD['y'].values, 
                 SID = OD['SPILL ID'].values, 
                 DATE_TIME = OD['start_date'].values, 
                 CAUSE = OD['SPILL CAUSE'].values,
                 CATEGORY = OD['SPILL_TYPE'].values,
                 VOLUME = OD['SPILL_VOL'].values,
                 MATERIAL = OD['MATERIAL_SEWER_PIPE'].values,
                 AGE = OD['AGE_SEWER_PIPE'].values,
                 DIAMETER = OD['DIAMETER_OF_SEWER'].values,
                 PLOT = OD[SMM].values)

other_source = ColumnDataSource(other_data)


In [6]:
# Calculate range of measurements for color map
L = min(OD[SMM])
print(str(SMM)+" (min) = "+str(L))
H = max(OD[SMM])
print(str(SMM)+" (max) = "+str(H))

# Create color map
    
if SMM == 'SPILL_TYPE' : 
    color_mapper = CategoricalColorMapper(palette = ['green', 'blue', 'red'], factors=['Category 1', 'Category 2', 'Category 3'])
elif SMM == 'SPILL_VOL' :
    palette = Magma256
    palette = palette[::-1] # reverse palette
    color_mapper = LogColorMapper(palette=palette, low=L, high=H)
elif SMM == 'DIAMETER_OF_SEWER' :
    palette = Magma256
    palette = palette[::-1] # reverse palette
    color_mapper = LinearColorMapper(palette=palette, low=L, high=H)
elif SMM == 'AGE_SEWER_PIPE':
    palette = Magma256
    palette = palette[::-1] # reverse palette
    color_mapper = LinearColorMapper(palette=palette, low=L, high=H)
elif SMM == 'surface_temp' : 
    palette = ['#5e4fa2', '#3288bd', '#66c2a5', '#abdda4', '#fee08b', '#fdae61', '#f46d43', '#d53e4f', '#9e0142'] #9-color Spectral11
    #6-color one: ['#3288bd', '#66c2a5', '#e6f598', '#fdae61', '#d53e4f', '#9e0142'] 
    color_mapper = LinearColorMapper(palette=palette, low=250, high=310)
else : 
    palette = Viridis256
    palette = palette[::-1] # reverse palette
    color_mapper = LinearColorMapper(palette=palette, low=L, high=H)


sm_surface (min) = 0.024919118732213967
sm_surface (max) = 0.5916668772697449


In [ ]:
# Set up plot
p = figure(plot_width=900, plot_height=600,
           x_axis_type="mercator", y_axis_type="mercator",
           toolbar_location="left", toolbar_sticky=False,
           match_aspect=True,
           tools = "pan, box_zoom, wheel_zoom, zoom_in, zoom_out")
p.title.text = "Sewer Overflow Events in California (marker shade = "+str(SMM)+")"
p.title.align = "center"
p.title.text_color = "black"
p.title.text_font_size = "14px"
p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'

# Add map tile underneath
p.add_tile(get_provider(Vendors.STAMEN_TERRAIN))

# Add glyphs ("Rain" on top of "Other")
p.square(x='x', y='y', 
         source=other_source,
         legend="Other", 
         color={'field': 'PLOT', 'transform': color_mapper},
         line_color='grey',
         fill_alpha=1, 
         size=12)
p.triangle(x='x', y='y', 
         source=rain_source,
         legend="Rain", 
         color={'field': 'PLOT', 'transform': color_mapper},
         line_color='grey',
         fill_alpha=1,
         size=15)
        
# Add hovertool
hover = HoverTool()
if SMAP == True :
    hover.tooltips=[("Date/Time", "@DATE_TIME"), ("Spill ID", "@SID"),  ("Cause", "@CAUSE"), (SMM, '@PLOT'), ('Spill Type', '@CATEGORY'), ('Spill Volume', '@VOLUME'), ("Diameter", '@DIAMETER'), ("Material", '@MATERIAL'), ("Age", '@AGE')]
else :
    hover.tooltips=[("Date/Time", "@DATE_TIME"), ("Spill ID", "@SID"),  ("Cause", "@CAUSE"), ('Spill Type', '@CATEGORY'), ('Spill Volume', '@VOLUME'), ("Diameter", '@DIAMETER'), ("Material", '@MATERIAL'), ("Age", '@AGE')]
p.add_tools(hover)

# Add color bar
if SMM == 'SPILL_TYPE' :
    citation = Label(x=10, y=10, x_units='screen', y_units='screen',
                 text='Category 1 = green; Category 2 = blue; Category 3 = red', render_mode='css',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    p.add_layout(citation)
else :
    color_bar = ColorBar(color_mapper=color_mapper, width=12, location=(0,0))
    p.add_layout(color_bar, 'right')

# Add legend with click-to-hide
p.legend.click_policy = 'hide'
p.legend.title = 'Cause of Spill'

# Show/output plot
filename="Spills+SMAP ("+str(SMM)+").html"
output_file(filename) 
output_notebook() 
show(p)




Loading BokehJS ...